# Climate Profiles 2

### Step 0: Set-Up
Import the [climakitae](https://github.com/cal-adapt/climakitae) library and other dependencies.

In [1]:
from typing import Tuple

import numpy as np
import pandas as pd
import xarray as xr
from tqdm.auto import tqdm  # Progress bar

import climakitae as ck
from climakitae.explore.standard_year_profile import get_climate_profile, export_profile_to_csv, _get_clean_standardyr_filename, retrieve_profile_data
from climakitae.explore.typical_meteorological_year import TMY
from climakitae.core.data_interface import (
    get_data_options,
    get_subsetting_options,
    get_data,
)

import warnings
warnings.filterwarnings("ignore")

# import functions needed for testing
from climakitae.core.constants import UNSET
from climakitae.core.paths import VARIABLE_DESCRIPTIONS_CSV_PATH
from climakitae.explore.typical_meteorological_year import is_HadISD
from climakitae.explore.standard_year_profile import (
    _get_clean_standardyr_filename,
    _check_stations,
    _check_cached_area,
    _check_lat_lon,
    match_str_to_wl,
)
from climakitae.util.utils import read_csv_file
from climakitae.util.warming_levels import get_gwl_at_year

In [2]:
variable = "Air Temperature at 2m"
units = "degF"
qtile = 0.5
warming_levels = [1.5]
no_delta = True

# warming level window size
valid_window = 5
invalid_window_1 = 2
invalid_window_2 = 5.5

# station name options
station_list = [
    "Sacramento Executive Airport (KSAC)",
    "Santa Barbara Municipal Airport (KSBA)",
]
station_name = ["Sacramento Executive Airport (KSAC)"]
custom_name = ["Custom Station Name"]
mixed_list = [
    "Custom Station Name",
    "Santa Barbara Municipal Airport (KSBA)",
]
custom_list = [
    "Custom Name 1",
    "Custom Name 2",
]

# lat lon
# latitude = 34.4041
# longitude = -121.5160 # no data found for these lat and lon bounds!
latitude = 34.19966
longitude = -118.36543  # no data found for these lat and lon bounds!

# cached area
area_name = "Los Angeles County"

### Time-based file modification

Valid inputs to test
- "approach" = "Time" with a "centered_year" between 2015 and 20199
- no "warming_level" input

Invalid inputs to test
1. "approach" = "Time" with a "centered_year" between 2015 and 2099, with "warming_level" input
2. "approach" = "Time" and no "centered_year" provided
3. "centered_year" outside of 2015-2099
4. "approach" that is not "Time" or "Warming Level"

In [3]:
profile_selections = {
    "variable": variable,
    "resolution": "3 km",
    "q": qtile,
    "warming_level": [1.5],
    "units": units,
    "no_delta": False,
    # # approach
    # "approach": "Time",
    # "centered_year": 2016,
    # # warming level window
    "warming_level_window": 5,
    # # Location options -- uncomment based on your desired location type
    # "stations": station_name,  # uncomment for a weather station
    # "latitude": (
    #     latitude - 0.02,
    #     latitude + 0.02,
    # ),  # uncomment for a using a custom coordinate location
    # "longitude": (
    #     longitude - 0.02,
    #     longitude + 0.02,
    # ),  # uncomment for a custom coordinate location
    "cached_area": area_name, # uncomment for a cached area
}

In [ ]:
profile = get_climate_profile(**profile_selections)

In [ ]:
export_profile_to_csv(profile, **profile_selections)

In [ ]:
name_params = {
    "var_id": "prec",
    "q": 0.75,
    "gwl": None,
    "location": "35-5N_122-5W",
    "no_delta": False,
    "warming_level_window": 5,
    "approach": "Time",
    "centered_year": 2016,
}

In [ ]:
_get_clean_standardyr_filename(**name_params)

### Return only bias-adjusted WRF models

The models are:

    "WRF_EC-Earth3_r1i1p1f1",
    "WRF_MPI-ESM1-2-HR_r3i1p1f1",
    "WRF_TaiESM1_r1i1p1f1",
    "WRF_MIROC6_r1i1p1f1",
    "WRF_EC-Earth3-Veg_r1i1p1f1",

#### Testing

In [3]:
profile_selections = {
    "variable": variable,
    "resolution": "3 km",
    #"q": qtile, # this is actually required for the export function
    #"warming_level": [1.5],
    "units": units,
    "no_delta": True,
    "bias_adjusted_models": False,
    # # approach
    "approach": "Time",
    "centered_year": 2016,
    "time_profile_scenario": "SSP 3-7.0",
    # # warming level window
    "warming_level_window": 5,
    # # Location options -- uncomment based on your desired location type
    # "stations": station_name,  # uncomment for a weather station
    # "latitude": (
    #     latitude - 0.02,
    #     latitude + 0.02,
    # ),  # uncomment for a using a custom coordinate location
    # "longitude": (
    #     longitude - 0.02,
    #     longitude + 0.02,
    # ),  # uncomment for a custom coordinate location
    "cached_area": area_name,  # uncomment for a cached area
}

In [4]:
profile = get_climate_profile(**profile_selections)

📊 Retrieving climate data...


Data retrieval:   0%|          | 0/1 [00:00<?, ?dataset/s]

You have chosen to produce a time-based Standard Year climate profile centered around 2016. 
Standard year functionality for time-based profiles: 
1. identifies the closest warming level at that centered year for either the input SSP scenario or default 'SSP 3-7.0' 
if no scenario input is provided. And 'historical' if 'centered_year' < 2015. 
2. then filters simulations by either the input SSP scenario or the default 'SSP 3-7-0'. 

Step 1: The corresponding global warming level for input centered year 2016 will now be determined.

Corresponding warming level for 'centered_year'= 2016 and SSP 'SSP 3-7.0' is [1.12]. 

Step 2: The climate profile will now be produced using SSP scenario 'SSP 3-7.0' and warming level [1.12]. Only models corresponding to this scenario will be used.
Using default 'q': 0.5
⚙️  Computing climate profiles...
      📊 Processing 87,600 hours (10 years) of data
      🎯 Computing 50th percentile for each hour of year
      ⚙️ Computing quantiles for 1 warming level

      Computing profiles:   0%|          | 0/8 [00:00<?, ?combo/s]

      ✅ Profile computation complete! Final shape: (365, 192)
         With index: Day of Year, columns: ['Hour', 'Simulation']
         Units: degF
   ✓ No baseline subtraction requested, returning raw future profile


In [5]:
profile

Hour                            1                               \
Simulation  CESM2-r11i1p1f1-ssp370 CNRM-ESM2-1-r1i1p1f2-ssp370   
Day of Year                                                      
Jan-01                   54.073235                   53.809341   
Jan-02                   56.273689                   50.876762   
Jan-03                   57.275036                   55.051403   
Jan-04                   55.488110                   55.830940   
Jan-05                   55.599838                   57.027294   
...                            ...                         ...   
Dec-27                   54.165520                   52.410072   
Dec-28                   56.153389                   51.450142   
Dec-29                   54.974113                   53.978039   
Dec-30                   56.998180                   50.833534   
Dec-31                   55.141823                   53.131210   

Hour                                                                 \
Simulation  EC-Earth3-r1i1p1f1-ssp370 EC-Earth3-Veg-r1i1p1f1-ssp370   
Day of Year                                                           
Jan-01                      50.047188                     54.390244   
Jan-02                      51.967049                     54.401619   
Jan-03                      54.235283                     55.066948   
Jan-04                      53.244865                     52.544655   
Jan-05                      51.981766                     51.616470   
...                               ...                           ...   
Dec-27                      52.579094                     51.995499   
Dec-28                      55.437572                     52.143105   
Dec-29                      51.080452                     52.820518   
Dec-30                      51.095226                     54.785919   
Dec-31                      50.770416                     53.527931   

Hour                                                          \
Simulation  FGOALS-g3-r1i1p1f1-ssp370 MIROC6-r1i1p1f1-ssp370   
Day of Year                                                    
Jan-01                      51.643112              56.933746   
Jan-02                      51.769295              58.184319   
Jan-03                      51.624107              56.279510   
Jan-04                      52.493568              54.767406   
Jan-05                      55.443726              56.140972   
...                               ...                    ...   
Dec-27                      54.842113              53.634277   
Dec-28                      55.814735              52.009895   
Dec-29                      54.823105              53.090782   
Dec-30                      52.377220              53.926292   
Dec-31                      50.742840              56.075493   

Hour                                                               \
Simulation  MPI-ESM1-2-HR-r3i1p1f1-ssp370 TaiESM1-r1i1p1f1-ssp370   
Day of Year                                                         
Jan-01                          51.095558               53.692558   
Jan-02                          51.671516               52.961693   
Jan-03                          51.879158               55.015808   
Jan-04                          52.768719               54.340698   
Jan-05                          54.440014               55.493217   
...                                   ...                     ...   
Dec-27                          52.837547               58.959076   
Dec-28                          54.393707               59.175064   
Dec-29                          52.328716               57.569084   
Dec-30                          51.908157               57.785683   
Dec-31                          50.205334               54.992737   

Hour                            2                               ...  \
Simulation  CESM2-r11i1p1f1-ssp370 CNRM-ESM2-1-r1i1p1f2-ssp370  ...   
Day of Year                                                     ...   
Jan

In [6]:
export_profile_to_csv(profile,**profile_selections) 

In [4]:
retrieve_selections = {
    "variable": variable,
    "resolution": "45 km",
    # "q": qtile,
    "warming_level": [1.5],
    "units": units,
    # "no_delta": False,
    "bias_adjusted_models": True,
    # # approach
    # "approach": "Time",
    # "centered_year": 2016,
    "time_profile_scenario": "SSP 2-4.5",
    # # warming level window
    "warming_level_window": 5,
    # # Location options -- uncomment based on your desired location type
    # "stations": station_name,  # uncomment for a weather station
    # "latitude": (
    #     latitude - 0.02,
    #     latitude + 0.02,
    # ),  # uncomment for a using a custom coordinate location
    # "longitude": (
    #     longitude - 0.02,
    #     longitude + 0.02,
    # ),  # uncomment for a custom coordinate location
    "cached_area": area_name,  # uncomment for a cached area
}

In [5]:
all_historic, all_future = retrieve_profile_data(**retrieve_selections)

ValueError: No bias-adjusted models available for 'time_profile_scenario' = SSP 2-4.5.

In [5]:
all_historic.coords['simulation'].to_dataframe()

,Lambert_Conformal,centered_year,simulation
simulation,,,
WRF_CESM2_r11i1p1f1_historical+ssp245,0,2019,WRF_CESM2_r11i1p1f1_historical+ssp245
WRF_CESM2_r11i1p1f1_historical+ssp370,0,2016,WRF_CESM2_r11i1p1f1_historical+ssp370
WRF_CESM2_r11i1p1f1_historical+ssp585,0,2016,WRF_CESM2_r11i1p1f1_historical+ssp585
WRF_CNRM-ESM2-1_r1i1p1f2_historical+ssp370,0,2022,WRF_CNRM-ESM2-1_r1i1p1f2_historical+ssp370
WRF_EC-Earth3_r1i1p1f1_historical+ssp370,0,2010,WRF_EC-Earth3_r1i1p1f1_historical+ssp370
WRF_EC-Earth3-Veg_r1i1p1f1_historical+ssp370,0,1995,WRF_EC-Earth3-Veg_r1i1p1f1_historical+ssp370
WRF_FGOALS-g3_r1i1p1f1_historical+ssp370,0,2013,WRF_FGOALS-g3_r1i1p1f1_historical+ssp370
WRF_MIROC6_r1i1p1f1_historical+ssp370,0,2029,WRF_MIROC6_r1i1p1f1_historical+ssp370
WRF_MPI-ESM1-2-HR_r3i1p1f1_historical+ssp370,0,2012,WRF_MPI-ESM1-2-HR_r3i1p1f1_historical+ssp370


All avaialable simulations:

WRF_CESM2_r11i1p1f1_historical+ssp245	
WRF_CESM2_r11i1p1f1_historical+ssp245

WRF_CESM2_r11i1p1f1_historical+ssp585
WRF_CESM2_r11i1p1f1_historical+ssp585


WRF_CESM2_r11i1p1f1_historical+ssp370
WRF_CESM2_r11i1p1f1_historical+ssp370
WRF_CNRM-ESM2-1_r1i1p1f2_historical+ssp370
WRF_CNRM-ESM2-1_r1i1p1f2_historical+ssp370
WRF_EC-Earth3_r1i1p1f1_historical+ssp370
WRF_EC-Earth3_r1i1p1f1_historical+ssp370
WRF_EC-Earth3-Veg_r1i1p1f1_historical+ssp370
WRF_EC-Earth3-Veg_r1i1p1f1_historical+ssp370
WRF_FGOALS-g3_r1i1p1f1_historical+ssp370
WRF_FGOALS-g3_r1i1p1f1_historical+ssp370
WRF_MIROC6_r1i1p1f1_historical+ssp370
WRF_MIROC6_r1i1p1f1_historical+ssp370
WRF_MPI-ESM1-2-HR_r3i1p1f1_historical+ssp370
WRF_MPI-ESM1-2-HR_r3i1p1f1_historical+ssp370
WRF_TaiESM1_r1i1p1f1_historical+ssp370	

bias-adjusted simulations:

